In [11]:
import matplotlib.pyplot as plt
import numpy as np
from sequential import Linear, MSELoss, Sequential
import pandas as pd

df = pd.read_csv('./data/housing.csv')
df = df.dropna()
df['ocean_proximity'] = df['ocean_proximity'].map({'NEAR BAY': 1, '<1H OCEAN': 2, 'INLAND': 3, 'NEAR OCEAN': 4, 'ISLAND': 5})

In [12]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [14]:
from sklearn.model_selection import train_test_split

X = df.drop('median_house_value', axis=1)
y = df['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)


In [15]:
X_train.shape
X_test.shape

(4087, 9)

In [17]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

prediction = lin_reg.predict(X_test)

mse = mean_squared_error(y_test, prediction)

print(f'MSE: {mse}')
print(f'RMSE: {mse ** 0.5}')

MSE: 4837107614.356031
RMSE: 69549.31785687068
